Here is a notebook for testing palindromes.

What is a palindrome?  Look at Wikipedia: https://en.wikipedia.org/wiki/Palindrome

Want lots of examples?  Look here: http://www.derf.net/palindromes/old.palindrome.html

In [1]:
def is_palindrome(test_string):
    """
    This function checks if the string is a palindrome
    """
    l=len(test_string)
    half_way=int(l/2) #This is half way through the string (what if l is odd?)
    index=0
    while (index<half_way and test_string[index]==test_string[l-index-1]):
        index=index+1
    return index==half_way #If it made it this far, it is a palindrome

In [2]:
test_strings=["racecar", "racingcar", "steponnopets", "oso", "hannah", "hello"]

In [3]:
for s in test_strings:
    print((s + " is ") + ("" if is_palindrome(s) else "not ") + "a palindrome" ) 

racecar is a palindrome
racingcar is not a palindrome
steponnopets is a palindrome
oso is a palindrome
hannah is a palindrome
hello is not a palindrome


In [4]:
more_test_strings=["the hat i take", "I like to drive my hot racecar to the park?", "the hairs of the rat are heavy", "the teller is prophetic"]

In [27]:
def clean_string(test_string):
    import re
    return re.sub('\W+','', test_string )

In [5]:
def brute_force_biggest_palindrome(test_string):
    """
    This function finds the biggest palindrome using a brute force search
    """
    #start by stripping whitespace and punctuation
    import re
    clean_string = re.sub('\W+','', test_string )
    l=len(clean_string)
    big_start=-1
    big_end=-1
    biggest=0
    #Iterate through all substrings and check for palindrome
    for i in range(0,l):
        for j in range (i+1, l):
            current_string=clean_string[i:j]
            if is_palindrome(current_string):
                if(len(current_string)>biggest):
                    biggest=len(current_string)
                    big_start=i
                    big_end=j
    return clean_string[big_start:big_end]   

In [6]:
for s in more_test_strings:
    print(brute_force_biggest_palindrome(s))

atita
otracecarto
heratareh
elle


In [7]:
def biggest_palindrome(test_string):
    """
    This function finds the biggest palindrome more efficiently
    """
    #start by stripping whitespace and punctuation
    import re
    clean_string = re.sub('\W+','', test_string )
    l=len(clean_string)
    candidates=[]
    biggest=0
    big_start=-1
    big_end=-1
    surviving=[]
    #Iterate through all characters in the string
    for i in range(0,l):
        #That character is a length 1 palindrome
        surviving=[i]
        #If the character is repeated, it makes a length 2 palindrome
        if clean_string[i-1]==clean_string[i]:
            surviving.append(i-1)
        for j in candidates:
            #Check if the palindrome substring [j:i-1] can be extended to [j-1:i] 
            if clean_string[j-1]==clean_string[i]:
                surviving.append(j-1)
            #If not, see if [j:i-1] is worth keeping
            else:
                if (i-j)> biggest:
                    biggest=i-j
                    big_start=j
                    big_end=i-1
        candidates=surviving
    #After reaching the end of the string, check all the candidates to see if they are the longest
    for j in candidates:
        if (l-j)> biggest:
                    biggest=l-j
                    big_start=j
                    big_end=l-1
    return clean_string[big_start:big_end+1]

In [8]:
for s in more_test_strings:
    print(biggest_palindrome(s))

atita
otracecarto
heratareh
elle


Manacher's algorthm needs to insert a dummy character between the characters of a string.  What is the most efficient way to do that in Python?  

In [9]:
def insert_dummy_chars(string):
    s=["a"]*(2*len(string))
    s[::2]=string
    s[1::2]="|"*(len(string))
    return ''.join(s)[0:-1]
def insert_dummy_chars_2(string):
    return ''.join([x+"|" for x in string])[0:-1]
def insert_dummy_chars_3(string):
    return ''.join([val for pair in zip(list(string), ["|"]*len(string)) for val in pair])[0:-1]

In [10]:
def test_method(method, test_string, print_result=False):
    #Used to test the performance of methods
    import time
    t0 = time.time()
    if print_result:
        print(method(test_string))
    method(test_string)
    t1 = time.time()
    print(t1-t0)

In [11]:
test_method(insert_dummy_chars, "hello"*1000000)
test_method(insert_dummy_chars_2, "hello"*1000000)
test_method(insert_dummy_chars_3, "hello"*1000000)

0.451198101044
0.697851896286
1.70982909203


In [12]:
def print_progress(s, c, i, r):
    #This method was used to debug manachers algorithm
    str=""
    for k in range(0,len(s)):
        l=2*i-r
        j=2*c-i
        if (k==l):
            str=str+"<"
        if (k==j):
            str=str+"{"
        if (k==i):
            str=str+"["
        if (k==c):
            str=str+"\""
        str=str+s[k]
        if (k==c):
            str=str+"\""
        if (k==i):
            str=str+"]"
        if (k==j):
            str=str+"}"
        if (k==r):
            str=str+">"
    print(str)

In [13]:
def manachers(test_string):
    """
    This function finds the biggest palindrome using Manacher's algorithm
    """
    #start by stripping whitespace and punctuation
    import re
    clean_string = re.sub('\W+','', test_string )
    csl=len(clean_string)
    L=2*csl-1
    chars=insert_dummy_chars(clean_string)
    lengths=[0]*L
    c=0 #This is the center of the right-most known palindrome
    r=0 #This is the right end of the right-most known palindrome
    max_len=0
    max_i=0
    #Go through all positions and find the lenght of the palindrome centered there
    for i in range(0,L):
        #By symmetry of palindromes, we can infers some lengths from their mirrors to the left
        if(i<r):
            lengths[i]=min(lengths[2*c-i], r-i)
        else: #we have gone beyond the limits of what's known
            r=i
        if (lengths[i]+i==r): #The palindrome might extend beyond r.
            while (2*i-r>=0) and (r<L-1) and (chars[r+1]==chars[2*i-r-1]) :
                r=r+1
                lengths[i]=lengths[i]+1
                #check if its the biggest yet found
                if lengths[i]>max_len:
                    max_len=lengths[i]
                    max_i=i
            c=i #reset the center
    return clean_string[(1+max_i-max_len)/2:1+(max_i+max_len)/2]

In [14]:
for s in more_test_strings:
    print(manachers(s))

atita
otracecarto
heratareh
elle


Define a very large string to test with.  And a method which generates test strings too.

In [15]:
big_test_string="ieosoieieiekdkiekwowokdiekieiekdidkghgitheihhhgiehwowooejhdoihaoabaoifoiqo\
fhfhghgheieheidhdodiewososdfhsdoifksdlkhfwslldiofijwoidkjfoiwjokdjkjflsdjfowjjjdjjjdjjsjjsjjd\
iieiidiiekwowkdhhfhfiehfidfhghghgheiehghghghhghghhehehghththghfwoshdfosdfofhfhfhhhhshflsjhffdd\
iririirifkrifkfirkkfkrkkfifffirkrifkrirkfirkkkvkrifkfoekfifkfkfrjgkfjgirjfkfirkfjgkfndkdnfjejff\
isisiisiieieiidiiwiwiisidieieisisiieieiisisieieisisieieisisiieieisieisisieieiisiidiieisiisieiisis\
iiiiiiiiiiiiiitiiiiiiiiiiittiiiiiiiiiiiiiiitiiiiiiiiiihiitiiriieiiriitiiiviisisisisidifisidfisidfi\
sskskslslslslsdfjwodjsdoifsdfjslkfsldkjflskdjfowejfkdjfowjkdodkfjowidfksdjfwdokfjowdijfkkskkskgh\
hggghhghfjghgjfhgjghhghhgjfhgjfhgjjfhggfhgjghfhfjkghgjfhgjfhhfhhgjfhgjfhfjghgjjfjfhgjfhfjghfjhdjfhg\
hghghghhghhghhghghhghhghhggghhgghhghhghhghhghhggghgghghhghhgghgjfhghhghhghghgjjghhghhghhghhggghgghg\
ghhghtyghtyhgythghhghgythgythgjfhgjythgjfythgjdhghghhgjfhgythgyrnfjgyhgjfhgythgjdhghghghfhgjrythgjfhgh\
hghhghtjgyghtjfhghtyghgjfhgythghghhghhgyghtytghghghghghhrythgjdhgyggggfggggggghghgggggghgggghggghggggg\
ggggghgggggghggghgghgghggggghgggghgghgghgythggghghgjfhfjghfjghgghghghgjghfjghghghurhgurjfsdjhfdsjodwhvs\
hhhhhhhhhhghhhhhhhhhhthhhhhhhhhhyhhhhhhhhhhhthhhhhhhhhhhhghhhhhhhhghhghghfhghthhghtuthgufhurhghfhfhfhf\
racecarracecareracecareiiiiiiiiiiiiiiiracecariiiiiiiiiiiiiiiiracecariiiiiiiiracecar\
fhfhghgheieheidhdodiewososdfhsdoifksdlkhfwslldiofijwxoidkjfoiwjokdjkjflsdjfowjjjdjjjdjjsjjsjjd\
iieiidiiekwowkdhhfhfiehfidfhghghgheiehghghghhghghxhehehghththghfwoshdfosdfofhfhfhhhhshflsjhffdd\
iririirifkrifkfirkkfkrkkfifffirkrifkrirkfirkkkxvkrifkfoekfifkfkfrjgkfjgirjfkfirkfjgkfndkdnfjejff\
isisiisiieieiidiiwiwiisidieieisisiieieiisixsieieisisieieisisiieieisieisisieieiisiidiieisiisieiisis\
iiiiiiiiiiiiiitiiiiiiiiiiittiiiiiiiiiixiiiiitiiiiiiiiiihiitiiriieiiriitiiiviisisisisidifisidfisidfi\
sskskslslslslsdfjwodjsdoifsdfjslkfxsldkjflskdjfowejfkdjfowjkdodkfjowidfksdjfwdokfjowdijfkkskkskgh\
hggghhghfjghgjfhgjghhghhgjfhgjfxhgjjfhggfhgjghfhfjkghgjfhgjfhhfhhgjfhgjfhfjghgjjfjfhgjfhfjghfjhdjfhg\
hghghghhghhghhghghhghhghhgggxhhgghhghhghhghhghhggghgghghhghhgghgjfhghhghhghghgjjghhghhghhghhggghgghg\
ghhghtyghtyhgythghhghgythxgythgjfhgjythgjfythgjdhghghhgjfhgythgyrnfjgyhgjfhgythgjdhghghghfhgjrythgjfhgh\
hghhghtjgyghtjfhghtyghxgjfhgythghghhghhgyghtytghghghghghhrythgjdhgyggggfggggggghghgggggghgggghggghggggg\
ggggghgggggghggghggxhgghggggghgggghgghgghgythggghghgjfhfjghfjghgghghghgjghfjghghghurhgurjfsdjhfdsjodwhvs\
hhhhhhhhhhghhhhhhxhhhhthhhhhhhhhhyhhhhhhhhhhhthhhhhhhhhhhhghhhhhhhhghhghghfhghthhghtuthgufhurhghfhfhfhf\
fhfhghgheieheidhdodiewoxxxsosdfhsdoifksdlkhfwslldiofijwoidkjfoiwjokdjkjflsdxxxjfowjjjdjjjdjjsjjsjjd\
iieiidiiekwowkdhhxxfhfiehfidfhghghgheiehghghghhghghhehehghththghfwoshdfosdfofhfhfhhhhshflsjhffdd\
iririirifkrifkfirkkfkrkkfifffirkrifkrirkfirkkkvkrifkxfoekfifkfkfrjgkfjgirjfkfirkfjgkfndkdnfjejff\
isisiisiieieiidiiwiwiisidieieisisiieieiisisieieisisxxxieieisisiieieisieisisieieiisiidiieisiisieiisis\
iiiiiiiiiiiiiitiiiiiiiiiiittiiiiiixiiiiiiiiitiiiiiiiixxiihiitiiriieiiriitiiiviisisisisidifisidfisidfi\
sskskslslslslsdfjwodjsdoifsdfjslkfxsldkjflskdjfowejfkdjfowjkdodkfjowidfksdjfwdokfjowdijfkkskkskgh\
hggghhghfjghgjfhgjghhghhgjfhgjfhgjxjfhggfhgjghfhfjkghgjfhgjfhhfhhgjfhgjfhfjghgjjfjfhgjfhfjghfjhdjfhg\
hghghghhghhghhghghhghhghhggghhxgghhghhghhghhghhggghgghghhghhgghgjfhghhghhghghgjjghhghhghhghhggghgghg\
ghhghtyghtyhgythghhghgythgythgjfxhgjythgjfythgjdhghghhgjfhgythgyrnfjgyhgjfhgythgjdhghghghfhgjrythgjfhgh\
hghhghtjgyghtjfhghtyghgjfhgytxhghghhghhgyghtytghghghghghhrythgjdhgyggggfggggggghghgggggghgggghggghggggg\
ggggghgggggghggghgghgghgggxgghgggghgghgghgythggghghgjfhfjghfjghgghghghgjghfjghghghurhgurjfsdjhfdsjodwhvs\
hhhhhhhhhhghhhhhhhhhhthhhhhxhhhhhyhhhhhhxhhhhhthhhhhhxhhhhhhghhhhhhhhghhghghfhghthhghtuthgufhurhghfhfhfhf"

In [16]:
len(big_test_string)

3733

In [17]:
import random
def rand_string(length, num_characters):
    alphabet="abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ1234567890"[0:num_characters]
    return ''.join([alphabet[random.randint(0,num_characters-1)] for i in range(0,length)])

In [18]:
big_test_string_2=rand_string(10000000,2) #Only two characters
big_test_string_3=rand_string(10000000,3) #Only three characters

In [19]:
test_method(brute_force_biggest_palindrome, big_test_string)
test_method(biggest_palindrome, big_test_string)
test_method(manachers, big_test_string)

5.29237699509
0.00473499298096
0.00600099563599


In [20]:
test_method(biggest_palindrome, big_test_string*10000)
test_method(manachers, big_test_string*10000)

34.5642850399
59.1887221336


The absolute worse case for my algorithm is when all the characters are the same.  My algorithm should be N^2 in this case, while Manacher's should be N.

In [21]:
magnifiers=[10,100,1000,10000]

In [22]:
for m in magnifiers:
    test_method(biggest_palindrome, "a"*m)

0.000154972076416
0.00323295593262
0.229878902435
19.0263850689


In [23]:
for m in magnifiers:
    test_method(manachers, "a"*m)

0.000200986862183
0.000355005264282
0.00484299659729
0.0198569297791


But even for random strings generated with only two characters, my algorithm still seems to win out.

In [24]:
test_method(biggest_palindrome, big_test_string_2)
test_method(manachers, big_test_string_2)

11.0111079216
17.6508998871


In [26]:
test_method(biggest_palindrome, big_test_string_3)
test_method(manachers, big_test_string_3)

8.87521314621
15.5454192162


In [28]:
test_method(clean_string, big_test_string_2)

NameError: global name 're' is not defined

In [29]:
clean_string("hello")

NameError: global name 're' is not defined